<a href="https://colab.research.google.com/github/Aaditya18SP/MSC_Data_Science/blob/windows/prac7_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.master("local[2]").appName("prac7_8").getOrCreate()


In [7]:
movies_data = spark.read.csv("/content/sample_data/u.item",sep="|")
movies_data = movies_data.select("_c0","_c1")
movies_data = movies_data.toDF("id","name")

movies_dict = {}
for rec in movies_data.collect():
  dic = rec.asDict()
  movies_dict[int(dic["id"])]= dic["name"]

broadcast_var = spark.sparkContext.broadcast(movies_dict)



In [8]:
ratings_data = spark.read.csv("/content/sample_data/u.data", sep="\t")

ratings_data = ratings_data.toDF("user","item","rating","timestamp")

def newRowMaker(row):
  dic = row.asDict()
  return Row(user = int(dic["user"]), item = int(dic["item"]), movie = broadcast_var.value[int(dic["item"])], rating = float(dic["rating"]), timestamp = dic["timestamp"])

new_rows = ratings_data.rdd.map(newRowMaker)
new_ratings_data = spark.createDataFrame(new_rows)
new_ratings_data.show()

+----+----+--------------------+------+---------+
|user|item|               movie|rating|timestamp|
+----+----+--------------------+------+---------+
| 196| 242|        Kolya (1996)|   3.0|881250949|
| 186| 302|L.A. Confidential...|   3.0|891717742|
|  22| 377| Heavyweights (1994)|   1.0|878887116|
| 244|  51|Legends of the Fa...|   2.0|880606923|
| 166| 346| Jackie Brown (1997)|   1.0|886397596|
| 298| 474|Dr. Strangelove o...|   4.0|884182806|
| 115| 265|Hunt for Red Octo...|   2.0|881171488|
| 253| 465|Jungle Book, The ...|   5.0|891628467|
| 305| 451|       Grease (1978)|   3.0|886324817|
|   6|  86|Remains of the Da...|   3.0|883603013|
|  62| 257| Men in Black (1997)|   2.0|879372434|
| 286|1014|Romy and Michele'...|   5.0|879781125|
| 200| 222|Star Trek: First ...|   5.0|876042340|
| 210|  40|To Wong Foo, Than...|   3.0|891035994|
| 224|  29|Batman Forever (1...|   3.0|888104457|
| 303| 785|     Only You (1994)|   3.0|879485318|
| 122| 387|Age of Innocence,...|   5.0|879270459|


In [10]:
# prac 8 recommendations
filtered_movies_data = movies_data.withColumn("id", movies_data.id.cast("int"))

from pyspark.ml.recommendation import ALS

ratings_data = ratings_data.withColumns({"user":ratings_data.user.cast("int"), "item": ratings_data.item.cast("int"), "rating": ratings_data.rating.cast("float")})

model = ALS(userCol="user", itemCol="item", ratingCol="rating").fit(ratings_data)

user_df = spark.createDataFrame([{"user":3}])
pred = model.recommendForUserSubset(user_df, 10)

pred_dict = pred.collect()[0].asDict()
recommendations = pred_dict["recommendations"]

rec_df = spark.createDataFrame(recommendations)

rec_df.join(filtered_movies_data, rec_df.item == filtered_movies_data.id).select("id","name","rating").show()


+----+--------------------+------------------+
|  id|                name|            rating|
+----+--------------------+------------------+
|1643|   Angel Baby (1995)| 4.519691467285156|
| 320|Paradise Lost: Th...| 4.452881813049316|
|1368|Mina Tannenbaum (...|  4.34311580657959|
|1463|    Boys, Les (1997)| 4.200453758239746|
| 205|       Patton (1970)|  4.12100076675415|
|1500|Santa with Muscle...| 4.028735160827637|
| 127|Godfather, The (1...|4.0062360763549805|
|  50|    Star Wars (1977)| 3.964315176010132|
| 183|        Alien (1979)|3.9204788208007812|
| 195|Terminator, The (...| 3.896592617034912|
+----+--------------------+------------------+

